In [9]:
import torch
from transformers import AutoTokenizer, EsmForMaskedLM
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

from Bio import SeqIO
import pandas as pd
import numpy as np
from time import perf_counter_ns
import matplotlib.pyplot as plt
import gc

SEQ_MAX_LEN = 8096

### CUDA/Torch GPU Setup

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 4060 Ti


## Convert FASTA file to pd.DataFrame for easier processing

In [11]:
def fasta_to_dataframe(fasta_file):
    records = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        # Extract the ID after '>tr'
        id_info = seq_record.id.split('|')[1] # Adjust the index based on your FASTA file format
        sequence = str(seq_record.seq)
        
        # Generate a random index within the range of the sequence length
        random_index = np.random.randint(0, min(len(sequence), SEQ_MAX_LEN))
        # Replace the character at the random index with '<mask>'
        masked_sequence = sequence[:random_index] + '<mask>' + sequence[random_index+1:]
        
        records.append([id_info, sequence, masked_sequence])
    
    # Create a DataFrame from the records
    df = pd.DataFrame(records, columns=['ID', 'Sequence', 'Masked_Sequence'])
    return df

## Create Dataset

In [12]:
fasta_df = fasta_to_dataframe("human_protein_seq/uniprotkb_proteome_UP000005640.fasta")

In [13]:
fasta_df.head()

,ID,Sequence,Masked_Sequence
0,A0A075B6G3,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...
1,A0A087WV00,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...
2,A0A087WZT3,MELSAEYLREKLQRDLEAEHVLPSPGGVGQVRGETAASETQLGS,MELSAEYLREKLQRDLEAEHVL<mask>SPGGVGQVRGETAASETQLGS
3,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...
4,A0A087X296,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPV<mask>PCCYYPCQ...


## Preparing your model and tokenizer

Now we load our model and tokenizer. If using GPU, use `model.cuda()` to transfer the model to GPU.

In [14]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
model = EsmForMaskedLM.from_pretrained("facebook/esm2_t6_8M_UR50D")

In [15]:
# Ensure the model is in evaluation mode
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(f"Running test on {fasta_df.shape[0]} sequences.")

# set up accuracy & latency info - leave this in the same code block so that it resets each time
accuracy = []
latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', 'get_mask_id', "get_predicted", "cmp_sequences"])
sequence_length = []
ctr = 0

for id, fasta_row in fasta_df.iterrows():
    # comment this out for optimal speed
    ctr += 1
    if ctr % 100 == 0:
        percentage = "{:.4f}".format(ctr / fasta_df.shape[0])
        print(f"Processing... {percentage}")
        # try to remove anything no longer needed on the GPU mem
        gc.collect()
    
    sequence_length.append(len(fasta_row["Sequence"]))
    latency_time_stamps = []
    
    # try to save as much memory as possible by reducing fragmentation
    torch.cuda.empty_cache()
    
    # load model to GPU
    model.to(device)

    # TIME LOG: start
    latency_time_stamps.append(perf_counter_ns())

    # Tokenize the input and move it to the correct device
    inputs = tokenizer(fasta_row["Masked_Sequence"], max_length=min(len(fasta_row["Masked_Sequence"]), SEQ_MAX_LEN), truncation=True, return_tensors='pt')
    
    # TIME LOG: tokenize
    latency_time_stamps.append(perf_counter_ns())
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # TIME LOG: move_to_gpu
    latency_time_stamps.append(perf_counter_ns())
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # TIME LOG: model
    latency_time_stamps.append(perf_counter_ns())

    # Retrieve index of <mask>
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1).item()
    
    # TIME LOG: get_mask_id
    latency_time_stamps.append(perf_counter_ns())
    
    predicted_token = tokenizer.decode([predicted_token_id], skip_special_tokens=True)
    
    # TIME LOG: get_predicted
    latency_time_stamps.append(perf_counter_ns())
    
    unmasked_text = fasta_row['Sequence']
    masked_text = fasta_row["Masked_Sequence"]
    mask_position = masked_text.find('<mask>')
    actual_token = unmasked_text[mask_position]
    
    # TIME LOG: cmp_sequences
    latency_time_stamps.append(perf_counter_ns())

    if predicted_token == actual_token:
        accuracy.append(1)
    else:
        accuracy.append(0)
        
    latency.loc[len(latency.index)] = latency_time_stamps

## Accuracy Log

In [ ]:
print("Correctly Predicted: " + str(sum(accuracy)))
print("Total Runs: " + str(len(accuracy)))
print("Accuracy: " +  str(sum(accuracy)/len(accuracy)))

## Latency/Time Log

In [ ]:
# drop headers
latency_diff = latency.diff().dropna()

latency_diff.plot(kind='bar', figsize=(10, 6))
plt.title('Time Stamps')
plt.xlabel('Columns')
plt.ylabel('Time Stamps')
plt.show()

In [ ]:
# Calculate total latency
total_latency = latency['cmp_sequences'] - latency['start']

# Merge the total latency with sequence_length DataFrame
lat_len_df = pd.DataFrame({ "Latency": total_latency, "Sequence_Length": sequence_length})

# Plotting sequence length vs total latency
lat_len_df.plot(x='sequence_length', y='total_latency', kind='scatter', figsize=(10, 6))
plt.title('Latency Graph of Sequence Length vs Total Latency')
plt.xlabel('Sequence Length')
plt.ylabel('Total Latency')
plt.show()